<h1 style="text-align:center"> Drexel University </h1>
<h2 style = "text-align:center"> College of Computing and Informatics</h2>
<h2 style = "text-align:center">INFO 323: Cloud Computing and Big Data</h2>
<h3 style = "text-align:center">Assignment 4</h3>
<h3 style = "text-align:center">Eugene Yakovlev</h3>
<h3 style = "text-align:center">May 27, 2020</h3>
<div style="text-align:center; border-style:solid; padding: 10px">
<div style="font-weight:bold">Due Date: Sunday, 05/31/2020</div>
This assignment counts for 15% of the final grade
</div>

### A. Assignment Overview
This assignment provides the opportunity for you to practice with Spark data analytics. 

### B. What to Hand In
	
Sumbit a completed this Jupyter notebook. 

### C. How to Hand In

Submit your Jupyter notebook file through the course website in the Blackboard Learn system.

### D. When to Hand In

1. Submit your assignment no later than 11:59pm in the due date.
2. There will be a 10% (absolute value) deduction for each day of lateness, to a maximum of 3 days; assignments will not be accepted beyond that point. Missing work will earn a zero grade.

### Note: All the programming must be done in Spark platform

## Question 1: Classification in Spark: Run the following cell to import relevant modules

In [117]:
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer

### a. Load the weather-samples.csv data into a DataFrame and prints the columns.

In [118]:
# load weather-samples.csv df
df = spark.read.option("inferSchema", "true").option("header", "true").csv("weather-samples.csv")
# print df columns
df.printSchema()

root
 |-- number: integer (nullable = true)
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)



### b. Defines the columns in the weather data that will be used for the decision tree classifier (**Done**).

In [119]:
featureColumns = ['air_pressure_9am','air_temp_9am','avg_wind_direction_9am','avg_wind_speed_9am',
        'max_wind_direction_9am','max_wind_speed_9am','rain_accumulation_9am',
        'rain_duration_9am']

### c. Drop unused column. We do not need the "number" column in our data, so remove it from the DataFrame.

In [120]:
# drop "number" column
df = df.drop("number")
# verify "number" column is dropped
df.printSchema()

df.count()

root
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)



1095

### d. Remove all rows with missing data.

In [121]:
# drop rows with missing values in any of the columns
df = df.filter(df.air_pressure_9am.isNotNull())
df = df.filter(df.air_temp_9am.isNotNull())
df = df.filter(df.avg_wind_direction_9am.isNotNull())
df = df.filter(df.avg_wind_speed_9am.isNotNull())
df = df.filter(df.max_wind_direction_9am.isNotNull())
df = df.filter(df.max_wind_speed_9am.isNotNull())
df = df.filter(df.rain_accumulation_9am.isNotNull())
df = df.filter(df.rain_duration_9am.isNotNull())
df = df.filter(df.relative_humidity_9am.isNotNull())
df = df.filter(df.relative_humidity_3pm.isNotNull())

### e. Print the number of rows and columns in the DataFrame.

In [122]:
# print num rows in df
print("Number of rows:", df.count())
# print num columns in df
print("Number of columns:", len(df.columns))

Number of rows: 1064
Number of columns: 10


### f. Use "Binarizer" function to create a target categorical variable from the "relative_humidity_3pm" column. Name this target variable as "label". Specifically, if the humidity at 3pm is less than 25%, then we want the categorical value to be 0, otherwise the categorical value should be 1. Use "24.99999" for the threshold argument. Use "relative_humidity_3pm" for the inputCol argument. Use "label" for the outputCol argument.

In [123]:
from pyspark.ml.feature import Binarizer

binarizer = Binarizer(threshold=24.99999, inputCol="relative_humidity_3pm", outputCol="label")

### g. Apply the above "binarizer" tranformation to the input DataFrame to create a new DataFrame named "binarizedDF".

In [124]:
binarizedDF = binarizer.transform(df)

### h. Look at the values of the first four rows in the new DataFrame.

In [125]:
# check that label worked
binarizedDF.select("relative_humidity_3pm", "label").show(4)

+---------------------+-----+
|relative_humidity_3pm|label|
+---------------------+-----+
|                36.16|  1.0|
|           19.4265968|  0.0|
|                14.46|  0.0|
|          12.74254735|  0.0|
+---------------------+-----+
only showing top 4 rows



### i. Use "VectorAssembler" to aggregate the features which will be used to make predictions into a single column. Use "featureColumns" defined at step b for the inputCols argument. Use "features" for the outputCol argument.

In [126]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")

### j. Apply the above assembler to the DataFrame binarizedDF to create a new DataFrame called "assembled" with the aggregated features in a column.

In [127]:
assembled = assembler.transform(binarizedDF)

### k. Split the DataFrame "assembled" into "train" and "test" by calling randomSplit(). Specify the ratios of the two sets as 80% and 20%. Set the seed number to be 12345. 

In [128]:
import random

random.seed(12345)
train, test = assembled.randomSplit([0.8, 0.2])

### l. Print the number of rows in the train and test DataFrames to check the sizes. **NOTE**: if you get values that don't match the ratio, it is most likely due to the cluster's CPU configuration. It is OK for this assignment.

In [129]:
trainCount = train.count()
testCount = test.count()

print("There are", trainCount, "in the train dataset.")
print("There are", testCount, "in the test dataset.")

There are 860 in the train dataset.
There are 204 in the test dataset.


### m. Create and train decision tree. Use "label" as the labelCol argument. Use "features" as the  featuresCol argument. 
Also, set:
* maxDepth = 5 - stopping criterion for tree induction based on maximum depth of tree
* minInstancesPerNode = 20 - stopping criterion for tree induction based on minimum number of samples in a node
* impurity = "gini" - the impurity measure used to split nodes

In [130]:
from pyspark.ml.classification import DecisionTreeClassifier

dtModel = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=5, minInstancesPerNode=20, impurity="gini")

### n. Fit the model

In [131]:
fittedDt = dtModel.fit(train)

### o. Make prediction using the test data set. Store the results in "predictions".

In [132]:
predictions = fittedDt.transform(test)

### p. Look at the values of "prediction" and "label" of the first ten rows in the predictions to see whether the prediction matches the input

In [133]:
predictions.select("prediction","label").show(10)

+----------+-----+
|prediction|label|
+----------+-----+
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
+----------+-----+
only showing top 10 rows



### q. Create an instance of MulticlassClassificationEvaluator to determine the accuracy of the predictions. Use "label" for the labelCol argument. Use "prediction" for the predictionCol argument. Use "weightedPrecision" for the metricName argument.

In [134]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [135]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")

### r. Compute the accuracy by calling evaluate().

In [136]:
accuracy = evaluator.evaluate(predictions)

### s. Print out the accuracy

In [137]:
accuracy

0.7918552036199095

### t. Display confusion matrix. The MulticlassMetrics class can be used to generate a confusion matrix of the classifier model. However, unlike MulticlassClassificationEvaluator, MulticlassMetrics works with RDDs of numbers and not DataFrames, so we need to convert our predictions DataFrame into an RDD. Let us try (**Done**):

In [138]:
preds = predictions.select("prediction", "label")

In [139]:
preds.show(2)

+----------+-----+
|prediction|label|
+----------+-----+
|       1.0|  1.0|
|       1.0|  1.0|
+----------+-----+
only showing top 2 rows



In [140]:
preds.rdd.take(2)

[Row(prediction=1.0, label=1.0), Row(prediction=1.0, label=1.0)]

### u. We can map the RDD to tuple to get an RDD of numbers (**Done**).

In [141]:
predsRdd = preds.rdd.map(tuple)

### v. Create an instance of MulticlassMetrics with the RDD as created at the previous step.

In [142]:
mcMetrics = MulticlassMetrics(predsRdd)

### w. Use the confusionMatrix() function to get a Spark Matrix. Convert the matrix to a Python Numpy array by calling toArray(), and transpose to view.

In [143]:
mcMetrics.confusionMatrix().toArray()

array([[75., 18.],
       [25., 86.]])

## Question 2 Implementing a Recommender Using Spark MLlib
This question uses a subset, movieslens.dat, of the Movielens dataset: https://grouplens.org/datasets/movielens/

The Movielens data set captures movie ratings by user, along with user and movie attributes, and can be used for collaborative filtering. 

The dataset used in this question contains 100,000 ratings by 943 users on 1,682 items, with each user having rated at least 20 movies. 

The data movielens.dat is a tab-delimited, newline terminated text file with the following structure:

user id	| item id | rating | timestamp

### a. First run the following cell to import relevant modules

In [144]:
from pyspark.mllib.recommendation import MatrixFactorizationModel, Rating
from pyspark.mllib.recommendation import ALS
from pyspark.sql import Row

### b. Load the movielens.dat data into an RDD

In [145]:
moviesRdd = spark.read.text("movielens.dat").rdd

### c. Show the first 5 rows of the data

In [146]:
moviesRdd.take(5)

[Row(value='196\t242\t3\t881250949'),
 Row(value='186\t302\t3\t891717742'),
 Row(value='22\t377\t1\t878887116'),
 Row(value='244\t51\t2\t880606923'),
 Row(value='166\t346\t1\t886397596')]

### d. Apply a sequence of map functions to the input RDD to create a new RDD called "ratings". The "ratings" RDD is a set of Rating objects. Each Rating object consists of three fields that correspond to the first three fields of the input RDD. 

In [147]:
ratings = moviesRdd.map(lambda l: l.value.split('\t'))\
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

### e. Train a collaborative filtering model by applying the imported ALS model to the "ratings" RDD. 
Set:
* rank = 10
* mumIterations = 10

In [148]:
rank = 10
numIterations = 10

model = ALS.train(ratings, rank, numIterations)

### f. Create a test data by using a map function to "ratings". The map function extracts the first two fields of the "ratings" RDD.

In [149]:
test = ratings.map(lambda r: (r[0], r[1]))

### g. Show the values of first 5 rows of the test data.

In [150]:
test.take(5)

[(196, 242), (186, 302), (22, 377), (244, 51), (166, 346)]

### h. Apply the model to the test data for prediction and store the results to "predictions".

In [151]:
predictions = model.predictAll(test)

### i. Prepare the preditions to join with true ratings (**Done**).

In [152]:
preds = predictions.map(lambda r: ((r[0], r[1]), r[2]))

In [153]:
preds.take(3)

[((195, 1084), 4.038826602567418),
 ((58, 1084), 3.8508604359832006),
 ((541, 1084), 4.293768907459729)]

### j. Join the predictions with the true ratings (**Done**).

In [154]:
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(preds)

In [155]:
ratesAndPreds.take(3)

[((196, 242), (3.0, 3.260025665083086)),
 ((186, 302), (3.0, 3.4392535752795967)),
 ((166, 346), (1.0, 1.945656412739603))]

### k. Compute and print the mean squared error from the RDD "ratesAndPreds". Specifically, compute the squared error for each pair of (rating, predition), sum up all the squared errors, and take the mean. The whole process can be done in one map function.

In [156]:
ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()

0.4826230341753457